##CARREGAR ARQUIVOS JSON


carregar os arquivos de dados JSON


entender o Schema gerado pelo arquivo

separa-los em fragmentos

criar uma visao com os dados para analisar dentro do SPARK.SQL

mostrar as frases com sentimentos negativos


In [1]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=439bf4e7d9f9c33a11511a60d1b892b668bc4706099dff6d6cf7963707bf037b
  Stored in directory: /root/.cache/pip/wheels/6f/db/e2/945dc8c17477cf9ab56d9f9aa1fa75b75335fffff9b12d867d
Successfully built spark


In [3]:
from pyspark.sql import SparkSession
sparksession = SparkSession.builder.appName("sentimento").getOrCreate()

In [5]:
path = "sentimento.json"
sentimentoDF =  sparksession.read.json(path, multiLine= "true")

In [6]:
sentimentoDF.show()

+--------------------+--------------------+
|             context|                 qas|
+--------------------+--------------------+
|spent the entire ...|[{[{55, my boss w...|
| oh! good idea ab...|[{[{5, good}], 25...|
|says good (or sho...|[{[{0, says good ...|
| i dont think you...|[{[{1, i dont thi...|
| haha better drun...|[{[{6, better}], ...|
|headache  wanna s...|[{[{0, headache}]...|
|had an awsome sal...|[{[{0, had an aws...|
| fine! going to d...|[{[{1, fine!}], a...|
| thank a yoou  ho...|[{[{1, thank}], a...|
|why don't adobe r...|[{[{0, why don't ...|
|prd take a long t...|[{[{0, prd take a...|
|_2008 well, havin...|[{[{1, 2008 well,...|
|    miss you my dear|[{[{1, miss you m...|
|have just bought ...|[{[{0, have just ...|
| ya mine too but ...|[{[{1, ya mine to...|
|today dan bought ...|[{[{107, , my tum...|
| oo noo thats not...|[{[{12, s not goo...|
|misses her phone....|[{[{20, having no...|
|so i have like no...|[{[{15, no more f...|
|i have perused th...|[{[{0, i h

In [7]:
sentimentoDF.printSchema()

root
 |-- context: string (nullable = true)
 |-- qas: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- answers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- is_impossible: boolean (nullable = true)
 |    |    |-- question: string (nullable = true)



In [10]:
sent2DF = sentimentoDF.select("context","qas.id", "qas.is_impossible","qas.question")
sent2DF.printSchema()

root
 |-- context: string (nullable = true)
 |-- id: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_impossible: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- question: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [11]:
sent2DF.show(10,False)

+------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+----------+
|context                                                                                                                             |id          |is_impossible|question  |
+------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+----------+
|spent the entire morning in a meeting w/ a vendor, and my boss was not happy w/ them. lots of fun.  i had other plans for my morning|[a3d0a7d5ad]|[false]      |[neutral] |
| oh! good idea about putting them on ice cream                                                                                      |[251b6a6766]|[false]      |[positive]|
|says good (or should i say bad?) afternoon!  http://plurk.com/p/wxpdj                                                               |[

In [12]:
sent3DF = sentimentoDF.select("context", "qas.question")
sent3DF.show()

+--------------------+----------+
|             context|  question|
+--------------------+----------+
|spent the entire ...| [neutral]|
| oh! good idea ab...|[positive]|
|says good (or sho...| [neutral]|
| i dont think you...|[negative]|
| haha better drun...|[positive]|
|headache  wanna s...|[negative]|
|had an awsome sal...|[positive]|
| fine! going to d...|[positive]|
| thank a yoou  ho...|[positive]|
|why don't adobe r...| [neutral]|
|prd take a long t...| [neutral]|
|_2008 well, havin...| [neutral]|
|    miss you my dear|[negative]|
|have just bought ...|[positive]|
| ya mine too but ...| [neutral]|
|today dan bought ...|[negative]|
| oo noo thats not...|[negative]|
|misses her phone....|[negative]|
|so i have like no...|[negative]|
|i have perused th...| [neutral]|
+--------------------+----------+
only showing top 20 rows



In [15]:
sent3DF.createOrReplaceTempView("sentimentos")

In [16]:
sparksession.sql("select context from sentimentos where context like '%hope%'").show()

+--------------------+
|             context|
+--------------------+
|get into me not g...|
| sorry to hear ab...|
| aww im sorry im ...|
|  hope you have a...|
|  is there going ...|
| you can find mor...|
|is predicting a h...|
| soon i hope... r...|
| aww hope uve had...|
| good evening ter...|
|just woke up, hop...|
| well sure hope t...|
|  that's no bueno...|
| i hope you had f...|
| i hope it's some...|
| oh no! hope you ...|
|yeha i broke thei...|
|back from brunch....|
|not feeling too g...|
| i hope you feel ...|
+--------------------+
only showing top 20 rows



In [20]:
sent4DF = sparksession.sql("select context, question, concat_ws(',',question)" + \
                         "as tps from sentimentos")

In [22]:
sent4DF.createOrReplaceTempView("tipo_sentimento")


In [24]:
sparksession.sql("select context, question from tipo_sentimento where tps = 'negative'").show()

+--------------------+----------+
|             context|  question|
+--------------------+----------+
| i dont think you...|[negative]|
|headache  wanna s...|[negative]|
|    miss you my dear|[negative]|
|today dan bought ...|[negative]|
| oo noo thats not...|[negative]|
|misses her phone....|[negative]|
|so i have like no...|[negative]|
|also i popped the...|[negative]|
|               uh oh|[negative]|
| what brody how d...|[negative]|
|i feel useless i ...|[negative]|
|kate is leaving m...|[negative]|
|i lost a follower...|[negative]|
| saying goodbye t...|[negative]|
|just got back in,...|[negative]|
|   my dog ran awayyy|[negative]|
|is missing someon...|[negative]|
| s'ok, trying to ...|[negative]|
|hot ****, i'm at ...|[negative]|
|just figured out ...|[negative]|
+--------------------+----------+
only showing top 20 rows



In [25]:
sparksession.sql("select tps, count(*) from tipo_sentimento group by tps ").show()

+--------+--------+
|     tps|count(1)|
+--------+--------+
|positive|    8582|
| neutral|   11117|
|negative|    7786|
+--------+--------+

